### You can click inside each cell and then click run or you can select everything and then click run.
  
### 

In [1]:
%config IPCompleter.greedy=True

In [2]:
from pyspark.sql import SparkSession

In [3]:
# You can auto-complete the code by pressing tab after writing 
# few letters of it. Example, type (without quotes) "SparkS" 
# then press tab.

spark = SparkSession.builder.appName("dataFrameExample").getOrCreate()

In [4]:
# Display the contents of sessoin. You can open the "Spkar UI" 
# link from output of this command in a new tab by right clicking on it.

spark

In [5]:
# We are downloading data from data.cityofnewyork.us 
# It contains baby names for NY, you can read more about it here
# https://data.cityofnewyork.us/Health/Popular-Baby-Names/25th-nujf

!wget -O babynames https://data.cityofnewyork.us/api/views/25th-nujf/rows.csv?accessType=DOWNLOAD

--2021-04-25 23:48:12--  https://data.cityofnewyork.us/api/views/25th-nujf/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘babynames’

babynames               [   <=>              ]   1.14M  2.02MB/s    in 0.6s    

2021-04-25 23:48:13 (2.02 MB/s) - ‘babynames’ saved [1192584]



In [6]:
# Lets create a DataFrame and while creating it, 
# we will rename some of the columns.

namesdf = spark.read.option("header", True) \
                    .option("inferSchema", True) \
                    .format("csv") \
                    .load("./babynames") \
                    .withColumnRenamed("Child's First Name", "name") \
                    .withColumnRenamed("Year of Birth", "birth_yr").cache()

In [7]:
# This action will read all data and then try to cache it in memory.

namesdf.count()

29464

In [8]:
# Lets check if dataframe is cached or not. True means its cached in memory.

namesdf.storageLevel.useMemory

True

In [9]:
namesdf.show(5)

+--------+------+---------+---------+-----+----+
|birth_yr|Gender|Ethnicity|     name|Count|Rank|
+--------+------+---------+---------+-----+----+
|    2011|FEMALE| HISPANIC|GERALDINE|   13|  75|
|    2011|FEMALE| HISPANIC|      GIA|   21|  67|
|    2011|FEMALE| HISPANIC|   GIANNA|   49|  42|
|    2011|FEMALE| HISPANIC|  GISELLE|   38|  51|
|    2011|FEMALE| HISPANIC|    GRACE|   36|  53|
+--------+------+---------+---------+-----+----+
only showing top 5 rows



In [10]:
# Lets look at how many years we have in this dataset using Spark functions. 

namesdf.select("birth_yr").distinct().show()

+--------+
|birth_yr|
+--------+
|    2015|
|    2013|
|    2014|
|    2012|
|    2016|
|    2011|
|    2017|
+--------+



In [11]:
# Lets create a view on this DataFrame so that we can run SQL.
# Yes, we love SQL, don't we :)

namesdf.createOrReplaceTempView("babynames")

In [12]:
spark.sql("""
select distinct ethnicity
from babynames
order by 1
""").show(20,False)

# Different values have the same meaning, data needs cleanup.

+--------------------------+
|ethnicity                 |
+--------------------------+
|ASIAN AND PACI            |
|ASIAN AND PACIFIC ISLANDER|
|BLACK NON HISP            |
|BLACK NON HISPANIC        |
|HISPANIC                  |
|WHITE NON HISP            |
|WHITE NON HISPANIC        |
+--------------------------+



In [13]:
# Clean and normalize ethnicity values with case when.

spark.sql("""

    select         
        case when ethnicity = 'ASIAN AND PACI' then 'ASIAN AND PACIFIC ISLANDER'
        when ethnicity = 'BLACK NON HISP' then 'BLACK NON HISPANIC'
        when ethnicity = 'WHITE NON HISP' then 'WHITE NON HISPANIC'
        else ethnicity
        end ethnicity,Gender, name, count(1) tot
    from babynames
        group by 1,2,3
    order by tot desc, name

""").show(10, False)

+--------------------------+------+--------+---+
|ethnicity                 |Gender|name    |tot|
+--------------------------+------+--------+---+
|BLACK NON HISPANIC        |FEMALE|Aaliyah |9  |
|HISPANIC                  |FEMALE|Aaliyah |9  |
|WHITE NON HISPANIC        |MALE  |Aaron   |9  |
|ASIAN AND PACIFIC ISLANDER|MALE  |Aaron   |9  |
|HISPANIC                  |MALE  |Aaron   |9  |
|BLACK NON HISPANIC        |MALE  |Aaron   |9  |
|ASIAN AND PACIFIC ISLANDER|FEMALE|Abby    |9  |
|ASIAN AND PACIFIC ISLANDER|MALE  |Abdullah|9  |
|WHITE NON HISPANIC        |FEMALE|Abigail |9  |
|WHITE NON HISPANIC        |FEMALE|Ada     |9  |
+--------------------------+------+--------+---+
only showing top 10 rows



In [14]:
# Lets look at most popular names for every Ethnicity, ignoring ties

spark.sql("""

select ethnicity,Gender, name, tot
from 
(
    select ethnicity,Gender, name, tot, row_number() over (partition by ethnicity, Gender order by tot desc) rnk
    from 
    (
        select 
        
        case when ethnicity = 'ASIAN AND PACI' then 'ASIAN AND PACIFIC ISLANDER'
        when ethnicity = 'BLACK NON HISP' then 'BLACK NON HISPANIC'
        when ethnicity = 'WHITE NON HISP' then 'WHITE NON HISPANIC'
        else ethnicity
        end ethnicity
        ,Gender, name, count(1) tot
        from babynames
         
        group by 1,2,3

    ) t
) 
where 
rnk  = 1
order by 1,2

""").show(100, False)

+--------------------------+------+-------+---+
|ethnicity                 |Gender|name   |tot|
+--------------------------+------+-------+---+
|ASIAN AND PACIFIC ISLANDER|FEMALE|Natalie|9  |
|ASIAN AND PACIFIC ISLANDER|MALE  |Kimi   |9  |
|BLACK NON HISPANIC        |FEMALE|Amina  |9  |
|BLACK NON HISPANIC        |MALE  |Kyle   |9  |
|HISPANIC                  |FEMALE|Aylin  |9  |
|HISPANIC                  |MALE  |Aaron  |9  |
|WHITE NON HISPANIC        |FEMALE|Jane   |9  |
|WHITE NON HISPANIC        |MALE  |Nicolas|9  |
+--------------------------+------+-------+---+



### Now you have an idea of how Spark and notebooks like Jupyter work, carry on!